In [1]:
from pianoq.misc.mplc_lab_imports import * 
%load_ext autoreload
%autoreload 2

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Camera things

In [44]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

# ml = MPLCAligner(use_cam=False)
# ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [50]:
cam.set_exposure_time(2)

In [47]:
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [48]:
mplc.load_masks(masks)
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [60]:
roi=(700, 300, 1250, 900)
name = f'tilts_{tilts}.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

In [61]:
cam.close()

# MPLC 

In [43]:
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

tilts = None 
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=tilts)
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_29_14_27_10_U1U.masks"  # with dead zone 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\U1U_2024_09_10_14_00_34.masks"  # old masks  

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
phases = np.random.uniform(0, 2*np.pi, size=50)
masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks)
# mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=None)
mplc.restore_location()

Got MPLC


# motors - actually now use in separate notebook

In [5]:
backlash = 0.1
wait_after_move = 0.3

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

Got Zaber motors
Got Thorlabs motors


In [131]:
mxi.move_absolute(9.30)
myi.move_absolute(4.05)
print(mxi.get_position())
print(myi.get_position())

9.3001
4.05


In [142]:
mxs.move_absolute(13.03)
mys.move_absolute(6.45)
print(mxs.get_position())
print(mys.get_position())

13.0300095
6.449996625


In [147]:
myi.move_relative(0.01)

In [4]:
print(mxi.get_position()) # 9.2701
print(myi.get_position()) # 3.3100

9.3
4.05


BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


## Scans to see what happened 

In [6]:
# Timetagger
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

### idler scan

In [8]:
integration_time = 1
start_x = 8.9
end_x = 9.6
start_y = 2.7
end_y = 5.2

mxi.wait_after_move = 0
myi.wait_after_move = 0
# start_x = 8.5
# end_x = 9.5
# start_y = 2
# end_y = 4

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 30 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_6_spots_again.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_idler_path)   
_ = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt)

### signal scan

In [9]:
integration_time = 1
start_x = 12.65
end_x = 13.35
start_y = 5.5
end_y = 8

mxs.wait_after_move = 0.1
mys.wait_after_move = 0.1

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 50 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_signal_path = fr'{dir_path}\{timestamp}_signal_initial_spots.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_signal_path)   
_ = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)

In [98]:
mimshow(scanner.single1s, vmax=3700)
mimshow(scanner.single2s, vmax=1000)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

# New WFM 

## with defocus distance and dead zone 

In [6]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_U1U.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_U1U.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=450e-6, last_plane_extra_dist=8.4e-3)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087 , 0.087 , 0.087 , 0.087 , 0.138 , 0.087 , 0.087 , 0.087 ,
       0.087 , 0.0954]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}
1


100%|██████████| 15/15 [02:45<00:00, 11.01s/it]


In [7]:
mplc_sim.res.show_overlap()

In [8]:
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9896838134848069

In [9]:
mplc_sim.res.show_all()

# mark spot locations 

In [11]:
from pianoq_results.scan_result import ScanResult
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
locs_signal_path = fr'{dir_path}\{timestamp}_s2_sig_mirror_3.5.locs'
locs_idler_path = fr'{dir_path}\{timestamp}_s1_idl_mirror_3.5.locs'

res_idl = ScanResult(fr'{dir_path}\2024_10_06_12_06_17_idler_3_5_spots.scan')
res_sig = ScanResult(fr'{dir_path}\2024_10_06_12_07_31_signal_3_5_spots.scan')
# res_idl.show_singles()
# res_sig.show_singles()
res_idl.get_xys(1, saveto_path=locs_idler_path)
res_sig.get_xys(2, saveto_path=locs_signal_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs
Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs


[(12.984470529470531, 7.019139194139194),
 (12.948240093240095, 6.8358558108558105),
 (12.937584082584083, 6.462895437895438),
 (12.897091242091243, 6.077147852147852),
 (12.869385614385616, 5.697793872793873)]

# Phases

In [20]:
# masks 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
# msks = MPLCMasks()
# msks.loadfrom(masks_path)
# masks = msks.real_masks
# masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
# mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=35)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_new_mirror.phases'
integration_time = 20

phases_result = PhaseFinderResult(fr"{dir_path}\2024_09_30_12_53_59_new_mirror.phases")


pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=phases_result.phases, coin_window=1e-9, no_hw_mode=True)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_sig = np.load(fr'{dir_path}\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs')['locs']
locs_idl = np.load(fr'{dir_path}\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs')['locs']

# i,j of corr matrix that is supposed to be strong 
i = 2
j = 2
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

0,0: 11142.8, 9004.2, 0.8
0,1: 11213.0, 8996.5, 0.7
0,2: 11140.5, 8981.6, 0.7
0,3: 11122.4, 8935.1, 0.9
0,4: 11145.1, 8958.0, 1.2
0,5: 11157.0, 8915.4, 1.8
0,6: 11137.5, 8623.6, 1.6
0,7: 11216.4, 8970.7, 2.1
0,8: 11235.2, 8999.5, 1.5
0,9: 11214.6, 9023.6, 1.8
1,0: 11191.5, 9026.8, 1.9
1,1: 11242.6, 9090.1, 2.2
1,2: 11253.8, 8921.4, 2.2
1,3: 11238.1, 8917.6, 2.8
1,4: 11276.6, 8962.9, 2.2
1,5: 11218.4, 8979.1, 1.8
1,6: 11243.7, 8949.0, 2.1
1,7: 11264.9, 8989.8, 2.0
1,8: 11229.2, 8994.8, 2.0
1,9: 11283.8, 8970.1, 2.9
2,0: 11278.7, 8794.3, 2.5
2,1: 11346.2, 8816.6, 2.5
2,2: 11298.0, 8890.1, 3.6
2,3: 11260.4, 8941.3, 3.5
2,4: 11277.9, 9039.8, 2.6
2,5: 11297.0, 8539.2, 2.4
2,6: 11316.2, 8312.1, 2.3
2,7: 11267.2, 8453.6, 2.5
2,8: 11325.0, 8847.4, 1.5
2,9: 11315.6, 8844.5, 1.2
3,0: 11298.9, 9008.6, 3.6
3,1: 11337.7, 8880.6, 3.0
3,2: 11334.7, 8639.1, 3.1
3,3: 11358.1, 8436.1, 2.4
3,4: 11313.4, 8518.9, 2.4
3,5: 11360.9, 8620.9, 1.8
3,6: 11330.2, 8823.6, 1.7
3,7: 11318.8, 8949.4, 1.5
3,8: 11399.5

# Correlations

In [25]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob
import time 

backlash = 0.0
wait_after_move = 0.3
integration_time = 4
coin_window = 1.2e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

scan_path = fr'{dir_path}\{timestamp}.dscan'

# locs_sig = np.load(fr'{dir_path}\2024_09_30_12_30_05_s2_sig_mirror.locs')['locs']
# locs_idl = np.load(fr'{dir_path}\2024_09_30_12_30_05_s1_idl_mirror.locs')['locs']

locs_sig = np.load(fr'{dir_path}\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs')['locs']
locs_idl = np.load(fr'{dir_path}\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs')['locs']

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)
# dps.zaber_ms = zaber_ms
dps.m_sig_x = mxs
dps.m_sig_y = mys
dps.m_idl_x = mxi
dps.m_idl_y = myi
dps.time_tagger = tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)

phases_result = PhaseFinderResult(fr"{dir_path}\2024_10_06_12_32_48_new_mirror.phases")
# phases_result = PhaseFinderResult(r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\U1U_2024_09_10_23_39_17.phases")

# masks 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
# msks = MPLCMasks()
# msks.loadfrom(masks_path)
# masks = msks.real_masks
# masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
# mplc.restore_location()
# masks = add_phase_input_spots(masks, phases_result.phases)
# mplc.load_masks(masks)
# 
# mplc.restore_location()

In [26]:
dps.scan()

beginning scan
0, 0: 356.50, 1244.75, 0.00
0, 1: 325.50, 6667.75, 0.00
0, 2: 343.25, 9003.00, 0.00
0, 3: 327.25, 5805.25, 0.00
0, 4: 352.75, 6687.75, 0.00
1, 0: 5168.00, 1209.50, 0.00
1, 1: 5282.50, 6632.75, 0.25
1, 2: 5237.25, 8984.50, 0.25
1, 3: 5203.00, 5784.50, 0.25
1, 4: 5280.00, 6687.25, 0.00
2, 0: 11787.25, 1260.00, 0.00
2, 1: 11615.00, 6614.50, 0.00
2, 2: 11757.00, 8972.50, 3.75
2, 3: 11722.25, 5843.50, 1.25
2, 4: 11758.25, 6721.75, 1.00
3, 0: 8071.25, 1252.75, 0.00
3, 1: 8114.00, 6574.25, 0.75
3, 2: 8060.75, 9023.75, 0.00
3, 3: 8165.25, 5787.50, 1.00
3, 4: 8133.25, 6751.00, 0.00
4, 0: 12292.25, 1218.00, 0.00
4, 1: 12375.00, 6523.25, 1.00
4, 2: 12384.25, 8984.25, 0.75
4, 3: 12416.00, 5827.50, 1.00
4, 4: 12361.25, 6671.50, 1.75


In [9]:
display(locs_idl)
display(locs_sig)

array([[9.25707792, 3.68295871],
       [9.29104396, 4.05432068],
       [9.30689477, 4.44606227],
       [9.34086081, 4.80383783],
       [9.36803363, 5.06650849]])

array([[12.6541342 ,  7.01487679],
       [12.94610889,  6.84651182],
       [12.93758408,  6.45863303],
       [12.89282884,  6.07714785],
       [12.86938561,  5.70205628]])

In [27]:
dps.res.show_singles()
dps.res.show()

In [34]:
mxi.move_absolute(9.3)
myi.move_absolute(3.90)
print(mxi.get_position())
print(myi.get_position())

9.30005
3.9


In [42]:
mxs.move_absolute(12.875)
mys.move_absolute(5.22)
print(mxs.get_position())
print(mys.get_position())

12.874990125000002
5.21998575
